In [1]:
import pandas as pd
import numpy as np
import spotipy
import plotly.express as px 

In [2]:
data = pd.read_csv("C://Users//marcs//OneDrive//Documents//Spotify//data//data.csv")
genre_data = pd.read_csv("C://Users//marcs//OneDrive//Documents//Spotify//data//data_by_genres.csv")
year_data = pd.read_csv("C://Users//marcs//OneDrive//Documents//Spotify//data/data_by_year.csv")
artist_data = pd.read_csv("C://Users//marcs//OneDrive//Documents//Spotify//data//data_by_artist.csv")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=25, 
                                   verbose=False))
                                 ], verbose=False)

X = data.select_dtypes(np.number)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)


MODELISATION

Sur la base de l'analyse et des visualisations, il est clair que les genres similaires ont tendance à avoir des points de données situés à proximité les uns des autres, tandis que des types de chansons similaires sont également regroupés ensemble.

Cette observation est logique. Les genres similaires auront des sonorités similaires et proviendront de périodes similaires, tout comme les chansons au sein de ces genres. Nous pouvons utiliser cette idée pour construire un système de recommandation en prenant les points de données des chansons qu'un utilisateur a écoutées et en recommandant des chansons correspondant à des points de données voisins.

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict



CLIENT_ID = "0c44dd32dc264ffc925263348f8df5f1"
CLIENT_SECRET = "33895e274f454993b57d91ccf2753e01"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, 
                                                           client_secret=CLIENT_SECRET))

In [5]:
'''
Trouver les détails des chansons dans l'ensemble de données de Spotify. 
Si la chanson n'est pas disponible dans l'ensemble de données, cela renvoie None.
'''
def find_song(name, year):
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
    if results['tracks']['items'] == []:
        return None
    
    
    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)

In [6]:
number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']

In [7]:
''''
Récupère les détails des chansons à partir de l'ensemble de données. 
Si les informations ne sont pas disponibles dans l'ensemble de données,
il recherchera les détails dans l'ensemble de données Spotify.

'''


def get_song_data(song, spotify_data):
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) 
                                & (spotify_data['year'] == song['year'])].iloc[0]
        print('Récupération des informations sur les chansons à partir d'un ensemble de données local')
        return song_data
    
    except IndexError:
        print('Récupération des informations sur les chansons à partir de l'ensemble de données Spotify')
        return find_song(song['name'], song['year'])

In [8]:
'''
Récupèrer les informations sur les chansons à partir de l'ensemble de données 
et calculer la moyenne de toutes les caractéristiques numériques des données des chansons
'''
def get_mean_vector(song_list, spotify_data):
    song_vectors = []
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Avertissement : {} n'existe pas dans Spotify ni dans la base de données'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))#nd-array où n est le nombre de chansons dans la liste. Il contient toutes les valeurs numériques des chansons dans une liste séparée.
    return np.mean(song_matrix, axis=0) 

In [9]:
'''
linéariser le dictionnaire en regroupant la clé et en formant une liste de valeurs pour la clé respective..
'''
def flatten_dict_list(dict_list):
    flattened_dict = defaultdict()
    for key in dict_list[0].keys(): 
        flattened_dict[key] = []
    for dic in dict_list:
        for key,value in dic.items():
            flattened_dict[key].append(value)
    return flattened_dict

In [10]:
'''
Mettre à l'échelle l'entrée moyenne ainsi que les caractéristiques numériques de l'ensemble de données.
Calculer la distance euclidienne entre l'entrée moyenne et l'ensemble de données.
Récupérer les 10 meilleures chansons avec la similarité maximale.

'''
import scipy
from scipy.spatial.distance import cdist



def recommend_songs( song_list, spotify_data, n_songs=10):
    
    metadata_cols = ['name', 'year', 'artists']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    
    scaler = song_cluster_pipeline.steps[0][1] 
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')

    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')

In [12]:
recommend_songs([{'name': 'Dior', 'year': 2019}],  data)

Fetching song information from spotify dataset


c:\Users\marcs\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': 'Hennessy and Moscato',
  'year': 2019,
  'artists': "['Shakur Luciano']"},
 {'name': 'Survival', 'year': 2018, 'artists': "['Drake']"},
 {'name': 'Perfect Time', 'year': 2019, 'artists': "['Roddy Ricch']"},
 {'name': 'No Lames (feat. Summer Walker)',
  'year': 2019,
  'artists': "['Kash Doll', 'Summer Walker']"},
 {'name': 'Nonchalant', 'year': 2018, 'artists': "['6LACK']"},
 {'name': 'I Sip', 'year': 2017, 'artists': "['Tory Lanez']"},
 {'name': 'Missing My Idols', 'year': 2018, 'artists': "['Trippie Redd']"},
 {'name': 'Growing Apart (To Get Closer)',
  'year': 2010,
  'artists': "['Kendrick Lamar', 'Jhené Aiko']"},
 {'name': 'II. No Exit', 'year': 2013, 'artists': "['Childish Gambino']"},
 {'name': 'November 18th', 'year': 2009, 'artists': "['Drake']"}]

In [13]:
recommend_songs([{'name': 'Bacc Seat', 'year': 2019}],  data)

Fetching song information from spotify dataset


c:\Users\marcs\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': 'Backseat Driver',
  'year': 2015,
  'artists': "['TobyMac', 'Hollyn', 'Tru']"},
 {'name': 'La Tortura (feat. Alejandro Sanz) - Alternate Version',
  'year': 2005,
  'artists': "['Shakira', 'Alejandro Sanz']"},
 {'name': 'Back It Up (feat. Jennifer Lopez & Pitbull) - Video Version',
  'year': 2015,
  'artists': "['Prince Royce', 'Jennifer Lopez', 'Pitbull']"},
 {'name': 'Sunshine', 'year': 2012, 'artists': "['Matisyahu']"},
 {'name': 'Ragoo', 'year': 2007, 'artists': "['Kings of Leon']"},
 {'name': 'World of Our Own - Single Remix',
  'year': 2011,
  'artists': "['Westlife']"},
 {'name': 'Dancing In The Dark - From The "Home" Soundtrack',
  'year': 2015,
  'artists': "['Rihanna']"},
 {'name': 'Violet', 'year': 2016, 'artists': "['Bad Suns']"},
 {'name': 'Santa No Soy',
  'year': 2004,
  'artists': "['RBD', 'Anahí', 'Dulce María', 'Maite Perroni', 'Christian Chávez', 'Christopher von Uckermann', 'Alfonso Herrera']"},
 {'name': 'Gonna Get This',
  'year': 2010,
  'artists': "['

In [15]:
recommend_songs([{'name': 'Body', 'year': 2020}],  data)

Fetching song information from local dataset


c:\Users\marcs\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': 'Best Part (feat. Daniel Caesar)',
  'year': 2017,
  'artists': "['H.E.R.', 'Daniel Caesar']"},
 {'name': 'Eternal Flame', 'year': 2004, 'artists': "['The Bangles']"},
 {'name': 'Perfectly Wrong', 'year': 2018, 'artists': "['Shawn Mendes']"},
 {'name': 'Love Me or Leave Me', 'year': 2015, 'artists': "['Little Mix']"},
 {'name': 'Sorry', 'year': 2017, 'artists': "['Halsey']"},
 {'name': 'Train Wreck - Acoustic',
  'year': 2020,
  'artists': "['James Arthur']"},
 {'name': 'Tum Hi Ho', 'year': 2013, 'artists': "['Arijit Singh']"},
 {'name': 'Vertigo', 'year': 2018, 'artists': "['Khalid']"},
 {'name': 'Comfortable', 'year': 2020, 'artists': "['H.E.R.']"}]

In [17]:
recommend_songs([{'name': 'Tadow', 'year': 2018}],  data)

Récupération des informations sur les chansons à partir d'un ensemble de données local


c:\Users\marcs\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': 'LIKE I WANT YOU', 'year': 2020, 'artists': "['Giveon']"},
 {'name': 'Diary (feat. Tony! Toni! Tone! & Jermaine Paul)',
  'year': 2003,
  'artists': "['Alicia Keys', 'Tony! Toni! Toné!', 'Jermaine Paul']"},
 {'name': 'Firestone', 'year': 2016, 'artists': "['Kygo', 'Conrad Sewell']"},
 {'name': 'Estamos Bien', 'year': 2018, 'artists': "['Bad Bunny']"},
 {'name': 'Nobody (feat. Athena Cage)',
  'year': 1996,
  'artists': "['Keith Sweat', 'Athena Cage']"},
 {'name': 'Hasta la Piel', 'year': 2012, 'artists': "['Carla Morrison']"},
 {'name': 'My Oasis (feat. Burna Boy)',
  'year': 2020,
  'artists': "['Sam Smith', 'Burna Boy']"},
 {'name': 'Spanish Guitar', 'year': 2000, 'artists': "['Toni Braxton']"},
 {'name': 'Steal',
  'year': 2015,
  'artists': "['Maribou State', 'Holly Walker']"}]

CONCLUSION

Nous sommes en mesure de recommander les 10 chansons les plus similaires à l'utilisateur en fonction de l'entrée fournie. La recommandation est basée sur la similarité des caractéristiques numériques des chansons. Nous avons calculé la distance cosinus et identifié les chansons présentant la plus grande similarité.